## Libraries + API Keys + Parameters

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
from crewai_tools import DirectoryReadTool, PDFSearchTool, FileWriterTool
from crewai import Agent, Crew, Process, Task
from crewai.tasks.task_output import TaskOutput

from langchain_openai import ChatOpenAI
from IPython.display import Markdown
import pandas as pd

/Users/mdashikadnan/Documents/adnanedu/python/ztm/crewai/crewai_venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [3]:
inputs = {
    'folder': 'invoices',
    'requirements': 'all invoices from Simonis and Braun',
    'columns': ['Date', "Description", 'Total Amount', 'Issuer', "Receiver"],
    'question' : 'what is the total amount invoiced and all the names of personnel involved in the latest invoice file'
}

## Assistant Agent - retrieve necessary documents

In [7]:
assistant_agent = Agent(
    role="Invoice Retrieval Assistant",
    goal=f"""
        Your task is to search the files within the folder **{inputs['folder']}** and locate the correct document based on the following criteria:
        **{inputs['requirements']}**
        Once identified, extract and retrieve the relevant data from the document.
    """,
    backstory="""
        You are an efficient and detail-oriented assistant specializing in organizing and retrieving relevant data from documents.
    """,
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8),  # Using GPT-4 model
    tools=[DirectoryReadTool(), PDFSearchTool()]
)

/Users/mdashikadnan/Documents/adnanedu/python/ztm/crewai/crewai_venv/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


In [8]:
assistant_task = Task(
    description=f"""
        Search for the correct file within **{inputs['folder']}** that matches the following criteria:
        **{inputs['requirements']}**
        Once found, extract all the data from the relevant file.
    """,
    expected_output="""
        The extracted data from the relevant file, formatted for clarity and completeness.
    """,
    agent=assistant_agent
)


## Organizer Agent - Format into CSV

In [9]:
organizer_agent = Agent(
    role="Data Organization Assistant",
    goal=f"""
        Process the provided data and format it as a CSV file with the specified columns:
        **{inputs['columns']}**
        Save the results strictly in CSV format without any additional text or formatting.
    """,
    backstory="""
        You are an efficient and detail-oriented assistant specializing in structuring and organizing data into CSV format.
    """,
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8),  # Using GPT-4 model
)


In [10]:
organizer_task = Task(
    description=f"""
        Format the provided data into a CSV file with the specified columns:
        **{inputs['columns']}**
        Ensure that all numeric values use a decimal point (.) instead of a comma (,).
        Date should be formatted as DD-MM-YYYY.
        Save the results strictly in CSV format without any additional text.
    """,
    expected_output="""
        A properly formatted CSV file containing the correct values in the specified columns.
    """,
    agent=organizer_agent,
)


## Analyst Agent - Answer query

In [11]:
analyst_agent = Agent(
    role="Data Analyst Assistant",
    goal=f"""
        Analyze the provided data and accurately answer the following question:
        **{inputs['question']}**
        Base your response strictly on data and factual analysis.
    """,
    backstory="""
        You are a detail-oriented data analyst, skilled at extracting insights and answering questions based on facts and data.
    """,
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8),  # Using GPT-4 model
)


In [12]:
analyst_task = Task(
    description=f"""
        Analyze the provided data and answer the following question:
        **{inputs['question']}**
        Ensure your response is based strictly on data-driven insights.
    """,
    expected_output="""
        A correct, data-backed answer along with the name of the file used for analysis.
    """,
    agent=analyst_agent,
)


## Assemble crew

In [13]:
crew = Crew(
    agents=[assistant_agent, organizer_agent, analyst_agent],  # List of agents involved
    tasks=[assistant_task, organizer_task, analyst_task],  # List of tasks to execute
    verbose=True,
    process=Process.sequential  # Ensuring tasks are processed in order
)
result = crew.kickoff()  # Start the task execution

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 012370ae-4fd1-4c2f-9faf-22cf8a93cc9f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: bc126282-f3dc-43b5-80d6-4536097275d9
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: bc126282-f3dc-43b5-80d6-4536097275d9
       Status: Executing Task...
    └── 🤖 Agent: Invoice Retrieval Assistant
            Status: In Progress

# Agent: Invoice Retrieval Assistant
## Task: 
        Search for the correct file within **invoices** that matches the following criteria:
        **all invoices from Simonis and Braun**
        Once found, extract all the data from the relevant file.
    


🤖 Agent: Invoice Retrieval Assistant
    Status: In Progress
└── 🧠 Thinking...

...

# Agent: Data Analyst Assistant
## Task: 
        Analyze the provided data and answer the following question:
        **what is the total amount invoiced and all the names of personnel involved in the latest invoice file**
        Ensure your response is based strictly on data-driven insights.
    


# Agent: Data Analyst Assistant
## Final Answer: 
The total amount invoiced in the latest invoice file, which is the Simonis Invoice dated 05.02.2025, is 188.402,52 €. The personnel involved are Garry Kilback (CEO of Simonis LLC) and Norwood Spinka (Customer).




🚀 Crew: crew
├── 📋 Task: bc126282-f3dc-43b5-80d6-4536097275d9
│      Assigned to: Invoice Retrieval Assistant
│      Status: ✅ Completed
│   └── 🤖 Agent: Invoice Retrieval Assistant
│           Status: ✅ Completed
├── 📋 Task: 1c4afdcd-ad5e-4ce2-b7ea-3a81891efc7f
│      Assigned to: Data Organization Assistant
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Organization Assistant
│           Status: ✅ Completed
└── 📋 Task: 856e7944-b0ab-4adc-af6a-59585180010e
       Status: Executing Task...
    └── 🤖 Agent: Data Analyst Assistant
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: bc126282-f3dc-43b5-80d6-4536097275d9
│      Assigned to: Invoice Retrieval Assistant
│      Status: ✅ Completed
│   └── 🤖 Agent: Invoice Retrieval Assistant
│           Status: ✅ Completed
├── 📋 Task: 1c4afdcd-ad5e-4ce2-b7ea-3a81891efc7f
│      Assigned to: Data Organization Assistant
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Organization Assistant
│           Status: ✅ Completed
└── 📋 Task: 856e7944-b0ab-4adc-af6a-59585180010e
       Assigned to: Data Analyst Assistant
       Status: ✅ Completed
    └── 🤖 Agent: Data Analyst Assistant
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 856e7944-b0ab-4adc-af6a-59585180010e                                                                     │
│  Agent: Data Analyst Assistant                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 012370ae-4fd1-4c2f-9faf-22cf8a93cc9f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [14]:
# raw data from the invoices
Markdown(result.tasks_output[0].raw)

Braun Invoice:
```
Date: 2022-10-04
Invoice No.: 31500
Customer: Mr. Cormier
Address: Stehr-Heaney, Donato Cormier, Leopold Forks, 711 24085 McCluremouth, Saint Pierre and Miquelon
Items:
1. Amount: 73 Pcs, Description: Quis ut excepturi quia odit., Price: 474,92 €, Total: 34.669,16 €
2. Amount: 79 Pcs, Description: Et qui voluptatem libero., Price: 554,69 €, Total: 43.820,51 €
3. Amount: 92 Pcs, Description: Accusamus ea autem., Price: 830,53 €, Total: 76.408,76 €
4. Amount: 67 Pcs, Description: Consectetur ratione explicabo., Price: 196,93 €, Total: 13.194,31 €
Price net: 168.092,74 €
Plus 10% VAT: 16.809,27 €
Invoice total: 184.902,01 €
Payment details: Payment within 90 days
CEO: Liliana Jast
Bank details: NL07LBKP8434276625 KPKMGASKW1P
```

Simonis Invoice:
```
Date: 05.02.2025
Invoice No.: 84951
Customer: Norwood Spinka
Address: Aurelia Wall 784, 42644-0051 South Maudieland
Company: Simonis LLC, Barney Village 891, 24163-6226 Kaseytown
Items:
1. Description: Deleniti consequatur hic omnis., Price: 918,80 €, Amount: 59, Total: 54.209,20 €
2. Description: Quis qui cum eligendi., Price: 162,67 €, Amount: 82, Total: 13.338,94 €
3. Description: Aut error voluptatem voluptatem., Price: 976,13 €, Amount: 82, Total: 80.042,66 €
4. Description: Assumenda nemo., Price: 171,00 €, Amount: 29, Total: 4.959,00 €
5. Description: Qui ab., Price: 309,57 €, Amount: 76, Total: 23.527,32 €
Price net: 176.077,12 €
Plus 7% VAT: 12.325,40 €
Invoice total: 188.402,52 €
Payment details: Payment within 30 days
CEO: Garry Kilback
Bank details: RS97378587727026913595 AADDWABV
```

In [15]:
# table structured
print(result.tasks_output[1].raw)

```
Date,Description,Total Amount,Issuer,Receiver
04-10-2022,Quis ut excepturi quia odit.,34669.16,Braun,Mr. Cormier
04-10-2022,Et qui voluptatem libero.,43820.51,Braun,Mr. Cormier
04-10-2022,Accusamus ea autem.,76408.76,Braun,Mr. Cormier
04-10-2022,Consectetur ratione explicabo.,13194.31,Braun,Mr. Cormier
05-02-2025,Deleniti consequatur hic omnis.,54209.20,Simonis LLC,Norwood Spinka
05-02-2025,Quis qui cum eligendi.,13338.94,Simonis LLC,Norwood Spinka
05-02-2025,Aut error voluptatem voluptatem.,80042.66,Simonis LLC,Norwood Spinka
05-02-2025,Assumenda nemo.,4959.00,Simonis LLC,Norwood Spinka
05-02-2025,Qui ab.,23527.32,Simonis LLC,Norwood Spinka
```


In [16]:
# write to csv
from io import StringIO

df = pd.read_csv(StringIO(result.tasks_output[1].raw))
df.dropna(inplace=True)
df.to_csv('result.csv')

In [17]:
# answer to the question
Markdown(result.tasks_output[2].raw)

The total amount invoiced in the latest invoice file, which is the Simonis Invoice dated 05.02.2025, is 188.402,52 €. The personnel involved are Garry Kilback (CEO of Simonis LLC) and Norwood Spinka (Customer).